In [ ]:
# imports
import json
from itertools import combinations_with_replacement
from random import choice

from rich import print
from tqdm import tqdm

In [ ]:
# inputs
# length of the stock board to be used [inches]
raw_board_length = 96
raw_board_count = 12

# define the lengths of cut boards needed in an array [(length [inches], count),]
cut_boards_requirements = [
    (19 + 7/8, 24),
    (16 + 9/16, 24),
    (13 + 9/16, 24),
    (8 + 7/16, 24),
]

# length of the saw blade [inches]
blade_length = 3/32

In [ ]:
# define the maximum number of boards each raw board can support
minimum_board_length = min([_[0] for _ in cut_boards_requirements])
maximum_supported_cuts = int(raw_board_length/minimum_board_length)

print(f"maximum_supported_cuts := {maximum_supported_cuts} using {minimum_board_length}")

In [ ]:
# find every viable method to cut the raw board into cut boards
cut_possibilities = list(combinations_with_replacement(list(range(len(cut_boards_requirements) + 1)), maximum_supported_cuts))

print(f"There exist {len(cut_possibilities)} potential cuts. Example: {choice(cut_possibilities)}")

In [ ]:
# determine the waste generated by each cut possibility
# while doing this remove cut configs that are too long
# cut possibilities analyzed
# [(cut_arrangement), ((number of board n cuts)), (blade waste), (raw waste), (total waste)]
cut_possibilities_analyzed = []
for cut_config in cut_possibilities:
    # create an empty array of cuts
    number_of_board_n_cuts = [0]*len(cut_boards_requirements)

    # iterate over each cut and count each type of board
    # also count the number of cuts
    cut_count = 0
    total_length_used = 0
    for cut in cut_config:
        # skip "no" cuts
        if cut == 0: continue

        # add to cut counts
        cut_count += 1

        # add to the number of boards tracker
        number_of_board_n_cuts[cut-1] += 1

        # add to the total length
        total_length_used += cut_boards_requirements[cut-1][0]

    # if the total length used exceeeds the raw baords skip
    # TODO: Consider skipping 0 waste for safety (> becomes >=)
    if total_length_used > raw_board_length: continue

    # calculate waste
    blade_waste = blade_length * (cut_count - 1)
    raw_waste = raw_board_length - total_length_used
    total_waste = blade_waste + raw_waste

    # append this analyzed configuration to the list
    cut_possibilities_analyzed.append([
        cut_config,
        number_of_board_n_cuts,
        blade_waste,
        raw_waste,
        total_waste
    ])

print(f"There exist {len(cut_possibilities_analyzed)} possible cuts. Example: {choice(cut_possibilities_analyzed)}")

In [ ]:
# get all possible arrangements of cuts
# add in the 0 case where the board is left untouched, this is a special case where we report 0 waste
inclusive_cut_possibilities_analyzed = cut_possibilities_analyzed.copy()
inclusive_cut_possibilities_analyzed.append([[0]*maximum_supported_cuts, [0]*len(cut_boards_requirements), 0, 0, 0])

# create the generator for combinations
all_possible_cut_operations = combinations_with_replacement(inclusive_cut_possibilities_analyzed, raw_board_count)

# define the operation to preform on a cut_operation
def check_cut_operation(cut_operation: list):
    # get just the cuts to be preformed
    cuts_list = [_[0] for _ in cut_operation]

    # get the waste values
    total_blade_waste = sum([_[2] for _ in cut_operation])
    total_raw_waste = sum([_[3] for _ in cut_operation])
    total_waste = total_blade_waste + total_raw_waste

    # sum the boards created by this operation set
    number_of_board_n_cuts = [0]*len(cut_boards_requirements)
    for cuts in cuts_list:
        for cut in cuts:
            if cut == 0: continue

            # add 1 to the number of board n cuts
            number_of_board_n_cuts[cut - 1] += 1

    # check if the right number of boards were created
    board_count_correct = True
    for i in range(len(cut_boards_requirements)):
        required = cut_boards_requirements[i]
        cut = number_of_board_n_cuts[i]

        if required != cut:
            board_count_correct = False
            break

    # skip of the board count is not correct
    if not board_count_correct: return None

    # print(cut_operation, total_blade_waste, total_raw_waste, total_waste, board_count_correct)
    return([cut_operation, total_blade_waste, total_raw_waste, total_waste])

In [ ]:
# mass calculation
# define results list
# allowed cut_operations [(cut operation), (total blade waste), (total raw waste), (total waste)]
allowed_cut_operations = []

# create a worker pool
for cut_operation in tqdm(all_possible_cut_operations):
    # apply and iterate over outputs
    response = check_cut_operation(cut_operation)
    if response: allowed_cut_operations.append({"operation": response})

In [ ]:
# save the allowed cut operations list as a json file
print("Saving allowed operations.")
with open("efficient_output.json", "w") as writer:
    json.dump(allowed_cut_operations, writer)
